# Visualize the trajectories of MD simulations and compute structural data
in particular, interatomic distances between C5 atoms of each nucleobase and hydrogen-bond formation

see also `importdata` notebook

In [1]:
import time, os
import numpy as np
import matplotlib.pyplot as plt
import pandas

In [2]:
import py3Dmol, nglview
import MDAnalysis as mda

In [3]:
dir_name = 'DATA_structures/'

if not os.path.exists(dir_name): os.mkdir(dir_name)

In [4]:
common_path = '/net/sbp/srnas2/tfrahlki/Simulations/%s_TREMD/Production/%s/'

## sketch of the data

The data are in directory `\textbf{/net/sbp/srnas2/tfrahlki/Simulations}`; let's use the 5 tetramers (AAAA, CAAU, CCCC, GACC, UUUU) and 2 hexamers (UCAAUC and UCUCGU) present there.

These data result from MD simulations performed with Temperature Replica Exchange (TRE); `T = 300 K` corresponds to n. 6. Inside these directories you find centered, preprocessed `.xtc` files which you could use to extract CVs. 

ATTENTION:
- TREMD simulation has been performed in a temperature range `275 K` to `400 K` so to analyze `300 K` I loaded all the files in the directories that you see when you do: `ls traj6*proc*` that have a numbering starting from 1 and ending at number >1

- analyzing the file: `traj6.'sequencename’_proc.xtc`, which is not the same as referred above, because it is the concatenated full trajectory; it will likely not work, because it exceed your RAM, so to analyze it load smaller subtrajectories and only concatenated the CVs that you want to analyze

- for the different systems the number of subtrajectories is different, so check inside each directory how many there areas a double check any of your analyzed CVs should exceed 1 million of frames, which corresponds to 1 microsecond

There are also: `reference_test.pdb` (the topology of the molecule, without solvent), `.mdp` files (?) (with the MD details), and the back-calculated observables `ls *.npy`)

So, to recap, for each tetramer (let's see for ex. AAAA), there are in the folder `Production/AAAA`:
- `traj6.AAAA.*proc.xtc` (the subtrajectories, unified in `traj6.AAAA_proc.xtc`)
- `reference_test.pdb` (the topology)
- `*.npy` (the back-calculated observables; in particular, 3J couplings, NOEs and uNOEs)

24 replicas (`ls md*.part0001.log`); replica exchange every 40 steps

regarding replex times: https://pubs.acs.org/doi/pdf/10.1021/ct100281c

Useful quantities to compute in order to assess the molecular structure:

- interatomic distances in order to assess nucleobase stacking (in particular, focus on C5 atoms); sometimes not all the 4 nucleobases are stacked on each other, they can also stack two and two
- planarity between the nucleobase rings

Read also:

`Conformational ensembles of RNA oligonucleotides from integrating NMR and molecular simulations`

`by Sandro Bottaro, Giovanni Bussi, Scott D. Kennedy, Douglas H. Turner, and Kresten Lindorff-Larsen`

In [5]:
sequence = 'UCUCGU'
path = common_path % (sequence, sequence)

In [6]:
subtraj_list = os.listdir(path)
subtraj_list = [s for s in subtraj_list if s.startswith('traj6.%s.' % sequence) and s.endswith('_proc.xtc')]

print('n. subtrajs: ', len(subtraj_list))

print('subtrajs: ', subtraj_list)

n. subtrajs:  5
subtrajs:  ['traj6.UCUCGU.1_proc.xtc', 'traj6.UCUCGU.2_proc.xtc', 'traj6.UCUCGU.3_proc.xtc', 'traj6.UCUCGU.4_proc.xtc', 'traj6.UCUCGU.5_proc.xtc']


## visualize structures
and save frames (or a movie of the trajectory)

The image from nglview.download_image() is not automatically saved to your filesystem by Python — instead, it triggers a download in your web browser. So:

- The image is saved where your browser usually saves downloads, e.g., your Downloads folder or whatever folder your browser is set to.

- It does not save the image to your server or Jupyter environment filesystem.

In [10]:
j = 0

# full trajectory is too long to visualize by nglview!
# univ = mda.Universe(path + 'reference_test.pdb', path + 'traj6.%s_proc.xtc' % sequence)

univ = mda.Universe(path + subtraj_list[j])

view = nglview.show_mdanalysis(univ)

# view.clear_representations()

# view.add_cartoon(selection='not hydrogen')
# view.add_representation('cartoon', selection='not hydrogen')
# view.add_cartoon(selection="nucleic and (name P or name O3' or name O5' or name C3' or name C4' or name C5')")

# view.add_ball_and_stick(selection='nucleic and not hydrogen')

view

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


NGLWidget(max_frame=90000)

In [184]:
j = 0

# full trajectory is too long to visualize by nglview!
# univ = mda.Universe(path + 'reference_test.pdb', path + 'traj6.%s_proc.xtc' % sequence)

univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])

view = nglview.show_mdanalysis(univ)

view.clear_representations()

# view.add_cartoon(selection='not hydrogen')
# view.add_representation('cartoon', selection='not hydrogen')
view.add_cartoon(selection="nucleic and (name P or name O3' or name O5' or name C3' or name C4' or name C5')")

view.add_ball_and_stick(selection='nucleic and not hydrogen')

view

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


NGLWidget(max_frame=90000)

In [185]:
img = view.download_image(filename="frame_%s_0.png" % sequence, factor=2, trim=True)
# critical issue: this command has to be run in a cell different from the one in which the view object is created!
# good thing: it saves the image exactly as you have modified it (rotation, translation, zoom) in the output
# of previous cell

## compute distances between C5 atoms of nucleobases

In [7]:
univ = mda.Universe(path + 'reference_test.pdb')
res_names = [res.resname for res in univ.residues]

res_names

['A', 'A', 'A', 'A']

In [8]:
# names = np.array([
#     ["A",'1',"C5", "A",'2',"C5"],
#     ['A','1','C5', "A",'3',"C5"]])

# for j in range(1):
for j in range(3, len(subtraj_list)):
# for j in range(len(subtraj_list)):

    distances = []
    counter = 0
    
    univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])
    # inn_counter=0
    
    for frame in univ.trajectory:
        # if not (inn_counter == 0):
        distances.append([])
        for i_resid in range(len(res_names)):
            for j_resid in range(i_resid + 1, len(res_names)):
                atom1 = univ.select_atoms('resid %i and name C5' % (i_resid + 1)).positions
                atom2 = univ.select_atoms('resid %i and name C5' % (j_resid + 1)).positions
                distances[counter].append(np.sqrt(np.sum(np.square(atom1 - atom2), axis=1)))
        counter += 1
        # inn_counter += 1
        if np.mod(counter, 1000) == 0: print(counter)
    
    distances = np.array(distances)[:, :, 0]

    print(distances.shape)

    np.save(dir_name + sequence + '_distances_%i.npy' % j, distances)

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/AAAA_TREMD/Production/AAAA/.traj6.AAAA.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/AAAA_TREMD/Production/AAAA/.traj6.AAAA.5_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

### repeat over different molecules

**BE CAREFUL** to correct sorting of subtrajectories!! check for AAAA and CAAU

In [7]:
sequences = ['UCUCGU']

for sequence in sequences:
    
    print(sequence)
    
    path = common_path % (sequence, sequence)

    subtraj_list = os.listdir(path)
    subtraj_list = [s for s in subtraj_list if s.startswith('traj6.%s.' % sequence) and s.endswith('_proc.xtc')]
    subtraj_list = np.sort(subtraj_list)

    univ = mda.Universe(path + 'reference_test.pdb')
    res_names = [res.resname for res in univ.residues]

    for j in range(len(subtraj_list)):

        distances = []
        counter = 0

        univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])

        for frame in univ.trajectory:
            distances.append([])
            for i_resid in range(len(res_names)):
                for j_resid in range(i_resid + 1, len(res_names)):
                    atom1 = univ.select_atoms('resid %i and name C5' % (i_resid + 1)).positions
                    atom2 = univ.select_atoms('resid %i and name C5' % (j_resid + 1)).positions
                    distances[counter].append(np.sqrt(np.sum(np.square(atom1 - atom2), axis=1)))
            counter += 1
            if np.mod(counter, 1000) == 0: print(counter)

        distances = np.array(distances)[:, :, 0]

        print(distances.shape)

        np.save(dir_name + sequence + '_distances_%i.npy' % j, distances)

UCUCGU


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
(90001, 15)


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

...

1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.4_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.5_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000
15

## Detect hydrogen-bond interactions

https://docs.mdanalysis.org/2.0.0/documentation_pages/analysis/hydrogenbonds.html

In [5]:
from MDAnalysis.analysis.hydrogenbonds.hbond_analysis import HydrogenBondAnalysis as HBA
# from MDAnalysis.analysis.hbonds import HydrogenBondAnalysis as HBA

In [6]:
hydrogens_sel = ("resname A and name H61 H62 or resname C and name H41 H42 or resname G and name H1 H21 H22 or "
    "resname U and name H3 or name HO2'")  # the last for non-canonical pairings mediated by sugar
donors_sel = ("resname A and name N6 or resname C and name N4 or resname G and name N1 N2 or "
    "resname U and name N3 or name O2'")
acceptors_sel = ("resname A and name N1 N3 N7 or resname C and name O2 N3 or resname G and name O6 N3 N7 or "
    "resname U and name O2 O4 or name O2'")

# or, specifically for UCAAUC:
# hydrogens_sel = "resname A and name H61 H62 or resname C and name H41 H42 or resname U and name H3"
# donors_sel = "resname A and name N6 or resname C and name N4 or resname U and name N3"
# acceptors_sel = "resname A and name N1 N3 N7 or resname C and name O2 N3 or resname U and name O2 O4"

In [9]:
for j in range(3, len(subtraj_list)):
    
    print('%i / %i' % (j + 1, len(subtraj_list)))

    univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])

    h = HBA(universe=univ, donors_sel=donors_sel, hydrogens_sel=hydrogens_sel, acceptors_sel=acceptors_sel,
            d_a_cutoff=3.5)

    h.run()
    
    np.save(dir_name + sequence + '_Hbonds_%i.npy' % j, h.results.hbonds)

3 / 5


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/AAAA_TREMD/Production/AAAA/.traj6.AAAA.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


4 / 5


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/AAAA_TREMD/Production/AAAA/.traj6.AAAA.5_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


### get n. of frames for each subtrajectory

In [ ]:
# read n. of frames for each subtrajectory

sequences = ['AAAA', 'CCCC', 'CAAU', 'GACC', 'UUUU', 'UCAAUC', 'UCUCGU']

n_frames = {}

for sequence in sequences:
    
    print(sequence)
    n_frames[sequence] = []
    
    path = common_path % (sequence, sequence)

    subtraj_list = os.listdir(path)
    subtraj_list = [s for s in subtraj_list if s.startswith('traj6.%s.' % sequence) and s.endswith('_proc.xtc')]
    subtraj_list = np.sort(subtraj_list)

    univ = mda.Universe(path + 'reference_test.pdb')
    res_names = [res.resname for res in univ.residues]

    for j in range(len(subtraj_list)):
        univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])
        n_frames[sequence].append(len(univ.trajectory))
        
n_frames

In [ ]:
import pickle

with open('n_frames.pickle', 'wb') as handle:
    pickle.dump(n_frames, handle, protocol=pickle.HIGHEST_PROTOCOL)

### repeat over different molecules

**PAY ATTENTION** to correct sorting of `subtraj_list` (check for `AAAA` oligomer)

In [ ]:
sequences = ['AAAA', 'CCCC', 'CAAU', 'GACC', 'UUUU', 'UCAAUC', 'UCUCGU']

for sequence in sequences:
    
    print(sequence)
    
    path = common_path % (sequence, sequence)

    subtraj_list = os.listdir(path)
    subtraj_list = [s for s in subtraj_list if s.startswith('traj6.%s.' % sequence) and s.endswith('_proc.xtc')]
    subtraj_list = np.sort(subtraj_list)

    univ = mda.Universe(path + 'reference_test.pdb')
    res_names = [res.resname for res in univ.residues]

    for j in range(len(subtraj_list)):

        distances = []
        counter = 0

        univ = mda.Universe(path + 'reference_test.pdb', path + subtraj_list[j])

        h = HBA(universe=univ, donors_sel=donors_sel, hydrogens_sel=hydrogens_sel, acceptors_sel=acceptors_sel,
            d_a_cutoff=3.5)

        h.run()
    
        np.save(dir_name + sequence + '_Hbonds_%i.npy' % j, h.results.hbonds)

CCCC


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CCCC_TREMD/Production/CCCC/.traj6.CCCC.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CCCC_TREMD/Production/CCCC/.traj6.CCCC.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CCCC_TREMD/Production/CCCC/.traj6.CCCC.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6

CAAU


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CAAU_TREMD/Production/CAAU/.traj6.CAAU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CAAU_TREMD/Production/CAAU/.traj6.CAAU.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/CAAU_TREMD/Production/CAAU/.traj6.CAAU.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6

GACC


...

/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UUUU_TREMD/Production/UUUU/.traj6.UUUU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UUUU_TREMD/Production/UUUU/.traj6.UUUU.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UUUU_TREMD/Production/UUUU/.traj6.UUUU.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6

UCAAUC


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCAAUC_TREMD/Production/UCAAUC/.traj6.UCAAUC.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCAAUC_TREMD/Production/UCAAUC/.traj6.UCAAUC.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCAAUC_TREMD/Production/UCAAUC/.traj6.UCAAUC.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.l

UCUCGU


/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.1_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.2_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))
/u/i/igilardo/.local/lib/python3.6/site-packages/MDAnalysis/coordinates/XDR.py:232: UserWarning: Couldn't save offsets because: [Errno 13] Permission denied: '/net/sbp/srnas2/tfrahlki/Simulations/UCUCGU_TREMD/Production/UCUCGU/.traj6.UCUCGU.3_proc.xtc_offsets.npz'
  warnings.warn("Couldn't save offsets because: {}".format(e))


### analyse the results of `HBA`

In [ ]:
print('results: \n(frame index, donor id, hydrogen id, acceptor id, distance, angle)\n', h.results.hbonds, '\n\n')

n_frames = h.results.hbonds.shape[0]

print('n. frames with H-bond interactions: ', n_frames)

frame_inds = np.int64(h.results.hbonds[:, 0])
donor_id = np.int64(h.results.hbonds[:, 1])
hydrogen_id = np.int64(h.results.hbonds[:, 2])
acceptor_id = np.int64(h.results.hbonds[:, 3])
distance = h.results.hbonds[:, 4]
angle = h.results.hbonds[:, 5]

frame_inds  

In [ ]:
i = 50

print("example: in frame %i \ndonor %i (%s) \nforms hydrogen-bond coupling with acceptor %i (%s)\nthrough exchange of hydrogen %i"
      % (frame_inds[i], donor_id[i], u.atoms[donor_id[i]], acceptor_id[i], u.atoms[acceptor_id[i]], hydrogen_id[i]))
print("at a distance %f and angle %f" % (distance[i], angle[i]))

In [ ]:
plt.plot(frame_inds, '.')

In [ ]:
print(donor_id, acceptor_id)

couples_don_acc = np.vstack((donor_id, acceptor_id))

couples = np.unique(couples_don_acc, axis=1)

print('\ndonor/acceptor couples:\n', couples)

plt.plot(donor_id, '.', label='donor')
plt.legend()

plt.figure()
plt.plot(acceptor_id, '.', label='acceptor')
plt.legend()

In [ ]:
for i in range(couples.shape[1]):
    print('couple: donor %s with acceptor %s' % (u.atoms[couples[0, i]], u.atoms[couples[1, i]]), '\n')